In [1]:
import pandas as pd
import re
import emoji
from underthesea import word_tokenize
import unicodedata
import json
from sklearn.model_selection import train_test_split

print(" Libraries installed and ready.")

d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\underthesea\pipeline\word_tokenize\regex_tokenize.py:66: SyntaxWarning: invalid escape sequence '\.'
  f"{UPPER}+(?:\.{W}+)+\.?",
d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\underthesea\pipeline\word_tokenize\regex_tokenize.py:66: SyntaxWarning: invalid escape sequence '\.'
  f"{UPPER}+(?:\.{W}+)+\.?",
d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\underthesea\pipeline\word_tokenize\regex_tokenize.py:71: SyntaxWarning: invalid escape sequence '\.'
  r"Mr\.", "Mrs\.", "Ms\.",
d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\underthesea\pipeline\word_tokenize\regex_tokenize.py:71: SyntaxWarning: invalid escape sequence '\.'
  r"Mr\.", "Mrs\.", "Ms\.",
d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\underthesea\pipeline\w

 Libraries installed and ready.


In [2]:
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load Train and Test
train_df = load_data('../data/train.jsonl')
test_df = load_data('../data/test.jsonl')

print(f"Original Train Shape: {train_df.shape}")
print(f"Original Test Shape: {test_df.shape}")

Original Train Shape: (1977, 2)
Original Test Shape: (494, 2)


In [3]:
# Dictionary to fix slang
teencode_dict = {
    "k": "không", "ko": "không", "hok": "không", "khong": "không", "kg": "không",
    "dc": "được", "đc": "được", "duoc": "được",
    "vn": "việt nam", "vnam": "việt nam",
    "ngân hàng": "ngân hàng", "nh": "ngân hàng",
    "tk": "tài khoản", "t khoản": "tài khoản",
    "dv": "dịch vụ",
    "add": "admin", "ad": "admin",
    "rep": "trả lời",
    "nt": "nhắn tin",
    "dt": "điện thoại", "sdt": "số điện thoại",
    "mn": "mọi người",
    "m": "mình", "mk": "mình", "mik": "mình",
    "bit": "biết", "bik": "biết",
    "gud": "tốt", "good": "tốt",
    "thanks": "cảm ơn", "tks": "cảm ơn", "thank": "cảm ơn", "iu": "yêu",
    "wá": "quá", "wa": "quá",
    "z": "vậy", "zậy": "vậy"
}

def replace_teencode(text):
    # Split text into words, replace if in dict, join back
    return " ".join([teencode_dict.get(word.lower(), word) for word in text.split()])

print(" Teencode Dictionary Loaded.")

 Teencode Dictionary Loaded.


In [4]:
def preprocess_text(text):
    # 0. Safety Check
    if not isinstance(text, str) or text.strip() == "":
        return ""
    
    # 1. Unicode Normalization
    text = unicodedata.normalize('NFC', text)
    
    # 2. Lowercase
    text = text.lower()
    
    # 3. Basic Cleaning (HTML, URLs)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    
    # 4. Word Elongation Fix (NEW)
    # Look for 3 or more repeated characters (e.g., "ngoooon") and replace with 1
    # We use \1 to refer to the captured character.
    text = re.sub(r'([a-zA-Z])\1{2,}', r'\1', text)
    
    # 5. Remove Duplicate Punctuation
    # "!!!!" -> "!"
    text = re.sub(r'([!?.])\1+', r'\1', text)
    
    # but for bank names, we usually just remove the '@' symbol.
    text = re.sub(r'[@#]', '', text)

    # 6. Demojize
    text = emoji.demojize(text, delimiters=(" :", ": "))
    
    # 7. Teencode Correction
    text = replace_teencode(text)
    
    # 8. Word Segmentation
    text = word_tokenize(text, format="text")
    
    return text.strip()

# Test the Elongation
print(preprocess_text("Dịch vụ tốt quá đi trờiiiiiii ơi !!!"))
# Expected: "dịch_vụ tốt quá đi trời ơi !"

dịch_vụ tốt quá đi trời_ơi !


In [5]:
# Define the mapping manually to ensure specific order
label_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

# Apply mapping
print("⏳ Encoding Labels...")
train_df['label'] = train_df['sentiment'].map(label_map)
test_df['label'] = test_df['sentiment'].map(label_map)

# Check if any label failed to map (sanity check)
if train_df['label'].isnull().sum() > 0:
    print(" WARNING: Some rows have unknown sentiments!")
    display(train_df[train_df['label'].isnull()])
    # Drop them if necessary
    train_df = train_df.dropna(subset=['label'])
    test_df = test_df.dropna(subset=['label'])
    
# Convert to integer (just to be safe)
train_df['label'] = train_df['label'].astype(int)
test_df['label'] = test_df['label'].astype(int)

print(" Labels Encoded: negative=0, neutral=1, positive=2")
display(train_df[['sentiment', 'label']].head())

⏳ Encoding Labels...
 Labels Encoded: negative=0, neutral=1, positive=2


,sentiment,label
0,negative,0
1,negative,0
2,negative,0
3,negative,0
4,negative,0


In [11]:
# --- 1. Apply Preprocessing ---
print(" Processing Train Data (Original)...")
train_df['text_clean'] = train_df['text'].apply(preprocess_text)

print(" Processing Test Data...")
test_df['text_clean'] = test_df['text'].apply(preprocess_text)

# --- 2. Clean & Deduplicate ---
# Drop duplicates (Smart Deduplication: Keep 1, drop rest)
train_df = train_df.drop_duplicates(subset=['text_clean'], keep='first')
test_df = test_df.drop_duplicates(subset=['text_clean'], keep='first')

# Drop empty rows (Garbage)
train_df = train_df[train_df['text_clean'].str.strip() != '']
test_df = test_df[test_df['text_clean'].str.strip() != '']

print(f"Rows after cleaning: {len(train_df)}")

# --- 3. Create Validation Split (NEW) ---
# We split the Cleaned Train data: 85% Train, 15% Validation.
# stratify=train_df['label'] ensures both sets have the same % of Negative/Positive classes.
print(" Splitting Train into Train & Validation sets...")
train_df, val_df = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df['label'], # Uses the 'label' column created in the previous cell
    random_state=42
)

# --- 4. Save to CSV ---
train_df.to_csv("../data/processed/train_processed.csv", index=False)
val_df.to_csv("../data/processed/val_processed.csv", index=False)
test_df.to_csv("../data/processed/test_processed.csv", index=False)

print("\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'")
print(f"• Final Train Shape: {train_df.shape}")
print(f"• Final Val Shape:   {val_df.shape}")
print(f"• Final Test Shape:  {test_df.shape}")

<>:35: SyntaxWarning: invalid escape sequence '\ '
<>:35: SyntaxWarning: invalid escape sequence '\ '
C:\Users\tntdu\AppData\Local\Temp\ipykernel_2528\4170370478.py:35: SyntaxWarning: invalid escape sequence '\ '
  print("\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'")


 Processing Train Data (Original)...
 Processing Test Data...
Rows after cleaning: 1542
 Splitting Train into Train & Validation sets...
\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'
• Final Train Shape: (1310, 4)
• Final Val Shape:   (232, 4)
• Final Test Shape:  (464, 4)


In [12]:
train_df.head()

,text,sentiment,label,text_clean
566,#tetyeuthuong chúc BIDV ngày càng phát triển :),positive,2,tetyeuthuong chúc bidv ngày_càng phát_triển :)
1728,"VCB dùng như...mứt, đi rút tiền 10 cây thì 7 c...",negative,0,"vcb dùng như . mứt , đi rút_tiền 10 cây thì 7 ..."
1520,Giao dịch viên chi nhánh nào xinh quá :o,positive,2,giao_dịch_viên chi_nhánh nào xinh quá : o
1106,X_60 + Y_10Make Heart luôn ạ <3 Chúc mừng 60 n...,positive,2,x_60 + y_10make heart luôn ạ_<3 chúc_mừng 60 n...
1658,Thank as nha,positive,2,cảm_ơn as_nha


In [13]:
val_df.head()

,text,sentiment,label,text_clean
1818,Chào vietcombank.vietcom ở phú yên.nằm ở đường...,negative,0,chào_vietcombank.vietcom ở phú_yên . nằm ở đườ...
141,cách thức chăm sóc khách hàng của hệ thống Ngâ...,negative,0,cách_thức chăm_sóc khách_hàng của hệ_thống ngâ...
1101,13 - 60 + 11 - 10 Chúc # BIDV # NganhangTMCPda...,positive,2,13 - 60 + 11 - 10 chúc bidv nganhangtmcpdautuv...
1490,Chúc mừng vietcombanh,positive,2,chúc_mừng vietcombanh
1259,FUCK DEP DM ĐUNG GUI TN SÔ DT KHUUÊN MÃI ĐÉO A...,negative,0,fuck dep dm đung_gui tn sô điện_thoại khuuên m...


In [14]:
test_df.head()

,text,sentiment,label,text_clean
0,Gọi k được mà tốn tiền như gì ấy,negative,0,gọi không được mà tốn tiền như gì ấy
1,"12h đêm thì sao nghẽn hả ad, mình cũng từng là...",negative,0,"12 h đêm thì sao nghẽn hả_ad , mình cũng từng ..."
2,Vietcombank ngân hàng tốt,positive,2,vietcombank ngân_hàng tốt
3,Chơi hoài mà không có trúng gì hết,neutral,1,chơi hoài mà không có trúng gì hết
4,Vietcom ít cây ATM quá :(,negative,0,vietcom ít cây atm quá : (
